In [143]:
import pandas as pd

In [144]:
#importing the CSV file into a DataFrame using Pandas
csv_file = pd.read_csv("winedata.csv")
csv_file.head(1)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia


In [145]:
#parsing out a year from the title & adding a new column consisting of years. 
##Refer back to "parsing_year" Jupyter Notebook file to see the vertification for this process.

title_list = [title for title in csv_file["title"]]

def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString) 

counter = 0
year_list = []
multiple_numbers_list = []
none_years_list = []
no_numbers_list = []

for title in title_list:
    title_strings_list = []
    title_numbers_list = []
    year_numbers_list = []
    for x in title.split():
        title_strings_list.append(x)
    if hasNumbers(title_strings_list) == True:
        for y in title_strings_list:
            if y.isdigit():
                title_numbers_list.append(int(y))
        year_numbers_list = [i for i in title_numbers_list if 1970 < i < 2019]
        if len(year_numbers_list) >= 1:
            year_list.append(year_numbers_list[0]) #All but one titles have the Year listed first. 
        #elif len(year_numbers_list) > 1:
            #multiple_numbers_list.append(counter)
        elif len(year_numbers_list) == 0:
            year_list.append("N/A")
    if hasNumbers(title_strings_list) == False:
        year_list.append("N/A")
    counter = counter + 1

year_list[102843] = 2000 #this one Title is an exception, refer back to "parsing_year" file for more info    

In [146]:
#adding the new list to the existing DataFrame. 

csv_file["year"] = year_list
csv_file_with_year = csv_file
print(f'Total number of wine review is: {len(csv_file_with_year["title"])}.')
csv_file_with_year.head(1)

Total number of wine review is: 129971.


,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,year
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013


In [147]:
#cleaning data
#1. Renaming the columns

df = csv_file_with_year.rename(columns={"Unnamed: 0": "Index", "country": "Country", "description":"Description",
                                       "designation": "Designation", "points":"Grade", "province": "Province",
                                        "region_1": "Region",
                                        "taster_name": "Taster", "taster_twitter_handle": "Taster_Twitter",
                                        "title": "Title", "variety": "Variety", "winery": "Winery", "year":"Year",
                                        "price": "Price"
                                       })
#1. Dropping designation, region_1, and region_2
df = df[["Index", "Title", "Variety", "Year", "Price", "Grade", "Winery", "Province", "Region", "Country", 
         "Taster", "Taster_Twitter", "Description"]]
#df.head(3)

#2. Removing all wines that do not have year assigned
df = df.loc[df["Year"]!= "N/A"]
#print(len(df["Year"]))

#3. Removing all wines that do not have Price tags
df = df.fillna("N/A")
df = df.loc[df["Price"] != "N/A"]
print(f'Total Number of Wine Reviews for dataframe with empty Price and year removed is: {len(df["Index"])}.')

#4. Removing Duplicates, if the same Taster reviewed the same wine more than once
df_duplicates_dropped = df.drop_duplicates(subset=["Title", "Price", "Grade", "Taster", "Description"], keep='first', inplace=False)
print(f'Total Number of Wine Reviews for dataframe after duplicates removed is: {len(df_duplicates_dropped["Index"])}.')
df_duplicates_dropped.head(1)

Total Number of Wine Reviews for dataframe with empty Price and year removed is: 116802.
Total Number of Wine Reviews for dataframe after duplicates removed is: 107734.


,Index,Title,Variety,Year,Price,Grade,Winery,Province,Region,Country,Taster,Taster_Twitter,Description
1,1,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,2011,15,87,Quinta dos Avidagos,Douro,N/A,Portugal,Roger Voss,@vossroger,"This is ripe and fruity, a wine that is smooth..."


In [148]:
#Exporting to a CSV file
new_df = df_duplicates_dropped

new_df = new_df.drop(columns=["Index"])

new_df.to_csv("winedata_edited.csv")

new_df.head(1)

,Title,Variety,Year,Price,Grade,Winery,Province,Region,Country,Taster,Taster_Twitter,Description
1,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,2011,15,87,Quinta dos Avidagos,Douro,N/A,Portugal,Roger Voss,@vossroger,"This is ripe and fruity, a wine that is smooth..."


In [149]:
#looking at the summary of the current dataset. 

number_of_unique_wine_reviews = len(new_df['Title'].unique())
number_of_unique_country = len(new_df['Country'].unique())
number_of_unique_province = len(new_df['Province'].unique())
number_of_unique_wineries = len(new_df['Winery'].unique())
number_of_unique_years = len(new_df['Year'].unique())
number_of_unique_taster = len(new_df['Taster'].unique())

summary = pd.DataFrame({"Total Number of Wine Reviews" : [number_of_unique_wines],
                    "Total Number of Wineries": [number_of_unique_wineries],
                   "Total Number of Provinces": [number_of_unique_province],
                   "Total Number of Countries": [number_of_unique_country],
                   "Total Number of Years" : [number_of_unique_years],
                   "Total Number of Tasters": [number_of_unique_taster]
})

summary.head(1)

,Total Number of Wine Reviews,Total Number of Wineries,Total Number of Provinces,Total Number of Countries,Total Number of Years,Total Number of Tasters
0,102771,15475,416,43,41,20


In [150]:
#Compiling the list of Countries. 

list_of_countries = new_df["Country"].value_counts()

list_of_countries

US                        49371
France                    15018
Italy                     14742
Spain                      5582
Portugal                   4273
Chile                      4106
Argentina                  3462
Austria                    2506
Australia                  2102
Germany                    1937
New Zealand                1232
South Africa               1180
Israel                      452
Greece                      406
Canada                      223
Bulgaria                    132
Hungary                     126
Romania                     102
Uruguay                      97
Turkey                       81
Georgia                      72
Slovenia                     69
Mexico                       68
Croatia                      67
N/A                          55
Moldova                      54
England                      47
Brazil                       35
Lebanon                      32
Morocco                      24
Peru                         16
Macedoni

In [151]:
#for i in list_of_countries:
    #print(i)
bottom_countries = []
    
for i, v in list_of_countries.items():
    if v < 1000:
        bottom_countries.append(i)
        
print(bottom_countries)
print(len(bottom_years))

['Israel', 'Greece', 'Canada', 'Bulgaria', 'Hungary', 'Romania', 'Uruguay', 'Turkey', 'Georgia', 'Slovenia', 'Mexico', 'Croatia', 'N/A', 'Moldova', 'England', 'Brazil', 'Lebanon', 'Morocco', 'Peru', 'Macedonia', 'Czech Republic', 'Cyprus', 'India', 'Serbia', 'Switzerland', 'Ukraine', 'Armenia', 'Bosnia and Herzegovina', 'Luxembourg', 'Slovakia', 'China']
28


In [152]:
new_df.head(1)

,Title,Variety,Year,Price,Grade,Winery,Province,Region,Country,Taster,Taster_Twitter,Description
1,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,2011,15,87,Quinta dos Avidagos,Douro,N/A,Portugal,Roger Voss,@vossroger,"This is ripe and fruity, a wine that is smooth..."


In [153]:
#Only including data from the top countries

for i in range(0, len(bottom_countries)):
    new_df = new_df.loc[new_df["Country"]!= bottom_countries[i]]

new_df.head(1)

,Title,Variety,Year,Price,Grade,Winery,Province,Region,Country,Taster,Taster_Twitter,Description
1,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,2011,15,87,Quinta dos Avidagos,Douro,N/A,Portugal,Roger Voss,@vossroger,"This is ripe and fruity, a wine that is smooth..."


In [154]:
#looking at the summary of the current dataset. 

number_of_unique_wine_reviews = len(new_df['Title'].unique())
number_of_unique_country = len(new_df['Country'].unique())
number_of_unique_province = len(new_df['Province'].unique())
number_of_unique_wineries = len(new_df['Winery'].unique())
number_of_unique_years = len(new_df['Year'].unique())
number_of_unique_taster = len(new_df['Taster'].unique())

summary = pd.DataFrame({"Total Number of Wine Reviews" : [number_of_unique_wines],
                    "Total Number of Wineries": [number_of_unique_wineries],
                   "Total Number of Provinces": [number_of_unique_province],
                   "Total Number of Countries": [number_of_unique_country],
                   "Total Number of Years" : [number_of_unique_years],
                   "Total Number of Tasters": [number_of_unique_taster]
})

summary.head(1)

,Total Number of Wine Reviews,Total Number of Wineries,Total Number of Provinces,Total Number of Countries,Total Number of Years,Total Number of Tasters
0,102771,15000,240,12,41,18


In [155]:
list_of_countries = new_df["Country"].value_counts()

remaining_countries = []
    
for i, v in list_of_countries.items():
    remaining_countries.append(i)
        
print(remaining_countries)

['US', 'France', 'Italy', 'Spain', 'Portugal', 'Chile', 'Argentina', 'Austria', 'Australia', 'Germany', 'New Zealand', 'South Africa']


In [156]:
list_of_years = new_df["Year"].value_counts()

list_of_years

2013    13434
2012    13152
2014    13120
2011    10266
2010    10093
2015     8382
2009     8314
2008     6228
2007     6077
2006     4892
2005     3131
2016     3124
2004     1512
2000      722
2001      653
1999      608
1998      533
2003      464
2002      318
1997      294
1996       59
1995       45
1994       23
1992       14
2017        9
1989        6
1990        5
1988        5
1985        4
1991        4
1986        3
1993        3
1987        2
1983        2
1978        2
1980        2
1984        2
1982        1
1973        1
1974        1
1976        1
Name: Year, dtype: int64

In [157]:
bottom_years = []
    
for i, v in list_of_years.items():
    if v < 1500:
        bottom_years.append(i)
        
print(bottom_years)
print(len(bottom_years))

[2000, 2001, 1999, 1998, 2003, 2002, 1997, 1996, 1995, 1994, 1992, 2017, 1989, 1990, 1988, 1985, 1991, 1986, 1993, 1987, 1983, 1978, 1980, 1984, 1982, 1973, 1974, 1976]
28


In [158]:
#Eliminating Years with less than 1,500 Wine reviews

for i in range(0, len(bottom_years)):
    new_df = new_df.loc[new_df["Year"]!= bottom_years[i]]

new_df.head(1)

,Title,Variety,Year,Price,Grade,Winery,Province,Region,Country,Taster,Taster_Twitter,Description
1,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,2011,15,87,Quinta dos Avidagos,Douro,N/A,Portugal,Roger Voss,@vossroger,"This is ripe and fruity, a wine that is smooth..."


In [159]:
#looking at the summary of the current dataset. 

number_of_unique_wine_reviews = len(new_df['Title'].unique())
number_of_unique_country = len(new_df['Country'].unique())
number_of_unique_province = len(new_df['Province'].unique())
number_of_unique_wineries = len(new_df['Winery'].unique())
number_of_unique_years = len(new_df['Year'].unique())
number_of_unique_taster = len(new_df['Taster'].unique())

summary = pd.DataFrame({"Total Number of Wine Reviews" : [number_of_unique_wines],
                    "Total Number of Wineries": [number_of_unique_wineries],
                   "Total Number of Provinces": [number_of_unique_province],
                   "Total Number of Countries": [number_of_unique_country],
                   "Total Number of Years" : [number_of_unique_years],
                   "Total Number of Tasters": [number_of_unique_taster]
})

summary.head(1)

,Total Number of Wine Reviews,Total Number of Wineries,Total Number of Provinces,Total Number of Countries,Total Number of Years,Total Number of Tasters
0,102771,14503,232,12,13,18


In [160]:
list_of_years = new_df["Year"].value_counts()

remaining_years = []
    
for i, v in list_of_years.items():
    remaining_years.append(i)
        
print(remaining_years)

[2013, 2012, 2014, 2011, 2010, 2015, 2009, 2008, 2007, 2006, 2005, 2016, 2004]


In [161]:
new_df.to_csv("winedata_edited.csv")

In [162]:
new_csv_file = pd.read_csv("winedata_edited.csv")

new_csv_file.head(1)

#looking at the summary of the current dataset. 

number_of_unique_wine_reviews = len(new_df['Title'].unique())
number_of_unique_country = len(new_df['Country'].unique())
number_of_unique_province = len(new_df['Province'].unique())
number_of_unique_wineries = len(new_df['Winery'].unique())
number_of_unique_years = len(new_df['Year'].unique())
number_of_unique_taster = len(new_df['Taster'].unique())

summary = pd.DataFrame({"Total Number of Wine Reviews" : [number_of_unique_wines],
                    "Total Number of Wineries": [number_of_unique_wineries],
                   "Total Number of Provinces": [number_of_unique_province],
                   "Total Number of Countries": [number_of_unique_country],
                   "Total Number of Years" : [number_of_unique_years],
                   "Total Number of Tasters": [number_of_unique_taster]
})

summary.head(1)

,Total Number of Wine Reviews,Total Number of Wineries,Total Number of Provinces,Total Number of Countries,Total Number of Years,Total Number of Tasters
0,102771,14503,232,12,13,18


In [115]:
new_df = new_df.loc[new_df["Country"]== "US"]

In [97]:
us_province = new_df["Province"].value_counts()
us_province

#drop 10 or below. 

California           32945
Washington            7843
Oregon                4881
New York              2377
Virginia               681
Idaho                  174
Michigan                99
Texas                   86
Colorado                60
America                 49
New Mexico              40
Arizona                 32
Missouri                24
North Carolina          22
Pennsylvania            17
Ohio                    10
New Jersey               8
Washington-Oregon        6
Massachusetts            5
Nevada                   3
Connecticut              3
Vermont                  2
Iowa                     2
Kentucky                 1
Rhode Island             1
Name: Province, dtype: int64

In [116]:
us_regions = new_df["Region"].value_counts()
len(us_regions)
us_regions

Napa Valley                                     3894
Columbia Valley (WA)                            3657
Russian River Valley                            2766
California                                      2072
Paso Robles                                     2068
Willamette Valley                               1997
Finger Lakes                                    1392
Sonoma Coast                                    1359
Sonoma County                                   1030
Walla Walla Valley (WA)                          983
Sta. Rita Hills                                  896
Santa Barbara County                             886
Carneros                                         871
Yakima Valley                                    858
Dry Creek Valley                                 828
Lodi                                             732
Santa Ynez Valley                                717
Santa Lucia Highlands                            714
Alexander Valley                              

In [24]:
#Looking to see if we can eliminate certain price tags

price_list = [price for price in df_duplicates_dropped["Price"] if 0 < price < 10]

d = {}
for item in price_list:
    if item in d:
        d[item] = d.get(item)+1
    else:
        d[item] = 1
        
for k,v in d.items():
    print(str(k)+':'+str(v))

9.0:1215
7.0:369
8.0:819
6.0:101
4.0:7
5.0:30


In [25]:
#df = df.loc[df["Year"]!= "N/A"]
four_dollars_wines = df_duplicates_dropped.loc[df_duplicates_dropped["Price"] == 4]

In [27]:
four_dollars_wines.head(2)

,Index,Title,Variety,Year,Price,Grade,Winery,Province,Country,Taster,Taster_Twitter,Description
1987,1987,Felix Solis 2013 Flirty Bird Syrah (Vino de la...,Syrah,2013,4,85,Felix Solis,Central Spain,Spain,Michael Schachner,@wineschach,Berry and cherry aromas are surprisingly sturd...
20484,20484,Dancing Coyote 2015 White (Clarksburg),White Blend,2015,4,85,Dancing Coyote,California,US,Jim Gordon,@gordone_cellars,"Fruity, soft and rather sweet, this wine smell..."


In [29]:
print(f'The new data set has {len(df_duplicates_dropped["Winery"].unique())} wineries.')

The new data set has 15475 wineries.


In [30]:
new_csv_file = pd.read_csv("winedata_edited.csv")

new_csv_file.head(1)

,Unnamed: 0,Index,Title,Variety,Year,Price,Grade,Winery,Province,Country,Taster,Taster_Twitter,Description
0,1,1,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,2011,15.0,87,Quinta dos Avidagos,Douro,Portugal,Roger Voss,@vossroger,"This is ripe and fruity, a wine that is smooth..."
